**Exercise 2: **
# Time and Frequency Domain

1. Compute the transient response of the system for a vertical force on the corner of the plate (choosing a forcing function which will show interesting dynamics)

2. Compute the steady state response for a vertical force at the corner in the frequency range of 2-40Hz

3. Visualize the response at characteristic frequencies

4. Plot the receptances, i.e. the transfer functions for the vertical excitation at the corner with respect to the displacement of the corner and the center of the plate

5. Plot the average vertical response of all points of the plate surface (one layer, e.g. top or bottom, is sufficient)

6. Estimate the repentance using the time domain data

7. Compare the receptance computed by the inversion of the dynamic stiffness matrix with the one computed from the model parameters (using the first few modes)

---

In [ ]:
import femtools
import importlib
importlib.reload(femtools)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks_cwt
from scipy import spatial
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 10)


## Build FEM System

In [ ]:
beam = femtools.FEMSys()

In [ ]:
B = femtools.IdxMasks(beam)

# select nodes on the west-side, i.e. at x=x_min
tol = 1e-12
x_min = beam.X[:,0].min()
Nw = np.argwhere(np.abs(beam.X[:,0]-x_min)<tol) # Node indices of West-Edge nodes
# select node on North-East-Top corner
Nnet = np.argwhere(np.all(np.abs(beam.X-beam.X.max(axis=0))<tol,axis=1))[0]
# select the data to plot (z-displacement of top nodes)
Nt = np.argwhere(np.abs(beam.X[:, 2]-beam.X[:, 2].max())<tol)

# indices of x, y, and z DoFs in the global system
# can be used to get DoF-index in global system, e.g. for y of node n by Iy[n]
Ix = np.arange(beam.nN)*3 # index of x-dofs
Iy = np.arange(beam.nN)*3+1
Iz = np.arange(beam.nN)*3+2

# select which indices in the global system must be constrained
If = np.array([Ix[Nw],Iy[Nw],Iz[Nw]]).ravel() # dof indices of fix constraint
Ic = np.array([(i in If) for i in np.arange(beam.nDof)]) # boolean array of constraind dofs

pt = [0, 0, 0]
distance, center_node = spatial.KDTree(beam.X).query(pt)


B.all = np.ones(beam.nDof, dtype=bool)
B.x = Ix
B.y = Iy
B.z = Iz
B.west = np.concatenate((Ix[Nw], Iy[Nw], Iz[Nw]))
B.net = np.concatenate((Ix[Nnet], Iy[Nnet], Iz[Nnet]))
B.top_z = Iz[Nt]
B.center_z = Iz[center_node]

## Harmonic Analysis

In [ ]:
beam.set_boundary(fix=B.west)
res_harmonic = beam.sim_freq(mode_count=15)

res_harmonic.animate(B.top_z, idx=0)

## 1. Transient Response
Compute the transient response of the system for a vertical force on the corner of the plate (choosing a forcing function which will show interesting dynamics)

In [ ]:
# Time stepping
dt = 0.003
T = 2
t = np.arange(dt,T,dt)

# Fix west edge
beam.set_boundary(fix=B.west)

# Force
Ts = 0.05
k = 4
impulse = np.exp(-(t/Ts)**k)
f = np.zeros((beam.nDof, len(t)))
f[B.z & B.net,:] = impulse

res_transient = beam.sim_time(t, force=f)

### North-East-Top Corner Displacement in z-Direction

In [ ]:
idx_array = np.argwhere((B.net & B.z)==True)
idx = idx_array[0, 0]
fig, ax = res_transient.plot_dof(dof=idx)

### FFT Analysis

In [ ]:
signal = res_transient.x[B.z & B.net,:]
freq, amp = femtools.fft(signal, res_transient.t)

fig,ax =plt.subplots()
ax.semilogy(freq, amp)
ax.set_xlabel('Freq in Hz')
ax.set_ylabel('displacement in m')
ax.set_xlim(0,60)

for xpos in res_harmonic.w0[:4]:
    plt.axvline(x=xpos, color='r', ls=':')

df = pd.DataFrame()
df['Natural Frequency w0'] = res_harmonic.w0[:4]
df

### 2. Steady State 
Compute the steady state response for a vertical force at the corner in the frequency range of 2-40Hz

In [ ]:
w_list = np.arange(2, 41)
x_center = np.zeros_like(w_list)

f = np.zeros(beam.nDof) 
f[B.net & B.z] = 1 # put force on North-East-Top corner

res_ss = beam.sim_steady_state(w_list, f)

### 3. Visualize the response at characteristic frequencies

In [ ]:
w_to_show = 5

w_dict = {i: x for x, i in enumerate(w_list)}
res_ss.animate(B.top_z, w_dict[w_to_show])

### 4. Plot the receptances
i.e. the transfer functions for the vertical excitation at the corner with respect to the displacement of the corner and the center of the plate

In [ ]:
tf_center = np.abs(res_ss.x[B.center_z, :].T) / 1
tf_net = np.abs(res_ss.x[B.net & B.z, :].T) / 1

fig,ax = plt.subplots()
ax.semilogy(w_list, tf_center, label='Center point')
ax.semilogy(w_list, tf_net, label='Corner')
ax.set_xlabel('Frequency w in Hz')
ax.set_ylabel('Receptance')
ax.legend()
for xc in res_harmonic.w0[:4]:
    plt.axvline(x=xc, color='r', ls=':')

### 5. Average Vertical Response
Plot the average vertical response of all points of the plate surface (one layer, e.g. top or bottom, is sufficient)

In [ ]:
avg_vertical = np.mean(np.abs(res_ss.x[B.top_z, :]), axis=0)

fig,ax = plt.subplots()
ax.semilogy(w_list, avg_vertical)
ax.set_xlabel('Frequency w in Hz')
ax.set_ylabel('Average vertical u in m')
for xc in res_harmonic.w0[:4]:
    plt.axvline(x=xc, color='r', ls=':')

### 6. Estimate the Receptence
Estimate the repentance using the time domain data

In [ ]:
from scipy import sparse

w_to_estimate = [5, 6, 25, 34, 35]
estimate = np.zeros(len(w_to_estimate))
idx = np.ix_(B.z & ~beam.Ic, B.net & B.z)

for i, w in enumerate(w_to_estimate):
    H = np.zeros((beam.nDof, beam.nDof), dtype=complex)
    
    Zw = beam.Kc + 1j*2*np.pi*w*beam.Cc - (2*np.pi*w)**2 * beam.Mc
    Hw = sparse.linalg.inv(Zw)
    H[np.ix_(~beam.Ic,~beam.Ic)] = Hw.toarray()
    
    estimate[i] = np.mean(np.abs(H[idx]))

### 7. Compare
Compare the receptance computed by the inversion of the dynamic stiffness matrix with the one computed from the model parameters (using the first few modes)

In [ ]:
fig,ax = plt.subplots()
ax.semilogy(w_list, avg_vertical, label='Simulation')
ax.scatter(w_to_estimate, estimate, label='Estimation')

ax.set_xlabel('Frequency w in Hz')
ax.set_ylabel('Receptance')
ax.legend()